In [31]:
import os
import pandas as pd
from glob import glob
from collections import defaultdict

In [34]:
root_path = '..\\..\\cache\\'
backtest_data = defaultdict(list)
for category in categories:
    category_path = root_path + f'{category}\\'
    csv_files = os.listdir(category_path)
    for file in csv_files:
        df = pd.read_csv(category_path + file, parse_dates=['date'])
        backtest_data[category].append(df)

In [ ]:
class Tester:
    
    def __init__(params)